In [ ]:
# %%

import pandas as pd

from source.utils import FREQ_DIR, RESULT_DIR, UCS_DIR, confirm_dir
from source.utils.associate import (BINARY_ASSOC_ARGS, add_extra_am,
                                    associate_ucs, confirm_basic_ucs)
from source.utils.associate import convert_ucs_to_csv as ucs2csv
from source.utils.associate import get_associations_csv as init_am, AM_DF_DIR
from source.utils.associate import manipulate_ucs, seek_readable_ucs, adjust_assoc_columns
pd.set_option('display.float_format', '{:,.2f}'.format)

/home/arh234/anaconda3/envs/dev-sanpi/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# %% [markdown]

 set parameters:
 - `UNIT` options:
   - `'Bigr'`
   - `'Adv'`
   - `'Adj'`
 - `PAT_DIR` options:
   - any directory in `results/freq_out/` containing a `ucs_format/` subdir with a `Trig*.tsv` frequency table
   - developed with the following in mind:
     - `POSmirror`
     - `NEGmirror`
     - `ANYmirror` (which has been populated by simple concatenation in the shell)

In [ ]:
UNIT = 'Adv'
# UNIT = 'Bigr' 
# UNIT = 'Adv'
# PAT_DIR = 'POSmirror'
PAT_DIR = 'POSmirror'
# PAT_DIR = 'ANYmirror'
FRQ_FLOOR = 3
# FRQ_FLOOR = 10
# FRQ_FLOOR = 20
# FRQ_FLOOR = 50
# FRQ_FLOOR = 100
TRIG_TSV = FREQ_DIR.joinpath(
    f'{PAT_DIR}/ucs_format/Trig{UNIT}_frq-thrMIN-7.35f.tsv')
FOCUS = ['f', 'unexpected_f',
         'conservative_log_ratio',
         'am_p1_given2', 'am_p2_given1',
         'am_p1_given2_simple', 'am_p2_given1_simple',
         'am_log_likelihood',
        #  'mutual_information', 'am_odds_ratio_disc', 't_score',
         'N', 'f1', 'f2', 'E11',
         'l1', 'l2']

In [ ]:
# %%

TRIGGER_POLARITY = {
    'positive': {'all',
                 'always',
                 'both',
                 'either',
                 'every',
                 'everybody',
                 'everyone',
                 'everything',
                 'many',
                 'often',
                 'or',
                 'some',
                 'somebody',
                 'someone',
                 'something',
                 'somethings',
                 'sometimes'},
    'negative': {'barely',
                 'hardly',
                 'neither',
                 'never',
                 'no',
                 'nobody',
                 'none',
                 'nor',
                 'nothing',
                 'seldom',
                 'rarely',
                 'scarcely'},

}

In [ ]:
# %%

TRIGGER_QUANT = {
    'existential': {
        'some',
        'somebody',
        'someone',
        'something',
        'somethings',
        'either',
     'or',
        'sometimes'
    },
    'universal': {
        'all', 'every',
        'always',
        'everybody',
        'everyone',
        'both',
        'everything',
    },
    'not_exist': {
        'neither',
        'never',
        'no',
        'nobody',
        'none',
        'nor',
        'nothing',
    },
    'hedged_not_exist': {
        'barely',
        'hardly',
        'rarely',
        'scarcely',
        'seldom',
        'few'
    },
    'hedged_universal':{
     'many',
     'often',
     }
}

In [ ]:
# %%

def invert_set_dict(d: dict):
    return {v: k for k in d for v in d[k]}

# %% [markdown]

## 1. Run `seek_readable_ucs()` to generate consistent output path

In [ ]:
readable = seek_readable_ucs(min_freq=FRQ_FLOOR,
                             ucs_subdir='trigger_eval',
                             contained_counts_path=TRIG_TSV)
print(readable.relative_to(RESULT_DIR))

    > seeking `trigger_eval/POSmirror/readable/TrigAdv_frq-thrMIN-7.35f_min3x*` frequency data and initial associations...
ucs/trigger_eval/POSmirror/readable/TrigAdv_frq-thrMIN-7.35f_min3x.rsort-view_am-only.csv


# %% [markdown]

 Snippet of frequencies to be analyzed (`TRIG_TSV`)

In [ ]:
! head -5 {TRIG_TSV} | column -t

64024  something  more
47547  all        more
45543  or         even
45382  all        too
40833  all        very


# %% [markdown]

## 2. Run `confirm_basic_ucs()`

In [ ]:
basic_ucs_path = readable.parent.parent.joinpath(
    readable.name.replace('.rsort-view_am-only.txt', '.ds.gz'))
print(f'Creating initial UCS table: `{basic_ucs_path.relative_to(RESULT_DIR)}')

basic_ucs_path = confirm_basic_ucs(
    basic_ucs_path,
    freq_floor=FRQ_FLOOR,
    contained_counts_path=TRIG_TSV)

Creating initial UCS table: `ucs/trigger_eval/POSmirror/TrigAdv_frq-thrMIN-7.35f_min3x.rsort-view_am-only.csv
+ existing UCS table found ✓


# %% [markdown]

 Excerpt of initial UCS table

In [ ]:
init_readable = UCS_DIR.joinpath(
    f'trigger_eval/{PAT_DIR}/readable'
).joinpath(f'{TRIG_TSV.name.replace(".tsv","")}_min{FRQ_FLOOR}x.init.txt')
! head -7 {init_readable}

         l1  l2                        f      f2      f1        N  
-----------  --------------------  -----  ------  ------  -------  
         or  more                  34344  207924  326614  1472077  
  something  more                  64024  207924  318343  1472077  
        all  more                  47547  207924  249279  1472077  
       some  more                  15949  207924  114929  1472077  
      often  more                  15169  207924   92233  1472077  


# %% [markdown]

## 3. Run `associate_ucs()`

In [ ]:
associate_ucs(basic_ucs_path)

transform_ucs_log = f'/share/compling/projects/sanpi/logs/associate/ucs//ucs-{PAT_DIR}_Trig{UNIT}_frq-thrMIN-7-35f_min{FRQ_FLOOR}x*.log'
! head -15 `ls -t1 {transform_ucs_log} | head -1`
! echo '...'
! tail -2 `ls -t1 {transform_ucs_log} | head -1`


Calculating UCS associations...

```
bash /share/compling/projects/sanpi/script/transform_ucs.sh /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigAdv_frq-thrMIN-7.35f_min3x.rsort-view_am-only.csv
> log will be saved to: /share/compling/projects/sanpi/logs/associate/ucs//ucs-POSmirror_TrigAdv_frq-thrMIN-7-35f_min3x.2024-05-25_2112.log
...
```

+ time elapsed → 00:00:13.10
# Manipulating TrigAdv_frq-thrMIN-7-35f_min3x ucs table
path to this script: /share/compling/projects/sanpi/script/transform_ucs.sh
Sat May 25 21:12:01 EDT 2024
(TMP: /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/tmp/tmp_POSmirror-20240525-211201.TrigAdv_frq-thrMIN-7-35f_min3x)
Caught a SIGPIPE at /usr/share/perl/5.30/Pod/Perldoc.pm line 1910.
Can't close STDOUT: Broken pipe at (eval 44) line 1.
END failed--call queue aborted.
## Initial Contingency Info

DATA SET FILE:  /share/compling/projects/sanpi/results/ucs/trigger_eval/POSmirror/TrigAdv_frq-thrMIN-7.35f_min3x.rsort-view_

# %% [markdown]

### Define dictionary containing relevant vocab sizes
 !!! Warning This is a `#HACK`: \
     Rather than developing a command/code to retrieve the vocab sizes programmatically,
     I simply copied the values given in the log output of `transform_usc.sh`
     for each `PAT_DIR`+`UNIT` combination

 |      |   ANYmirror |   NEGmirror |   POSmirror |
 |:-----|------------:|------------:|------------:|
 | Bigr |     395,338 |      64,644 |     330,694 |
 | Adv  |      23,125 |       5,004 |      18,121 |
 | Adj  |      83,422 |      21,562 |      61,860 |

In [ ]:
VOCABS = {'ANYmirror': {'Adv': 23125, 'Bigr': 395338, 'Adj': 83422},
          'NEGmirror': {'Adv': 5004, 'Bigr': 64644, 'Adj': 21562},
          'POSmirror': {'Adv': 18121, 'Bigr': 330694, 'Adj': 61860}
          }  # HACK
VOCAB = VOCABS[PAT_DIR][UNIT]
pd.DataFrame(VOCABS)

,ANYmirror,NEGmirror,POSmirror
Adv,23125,5004,18121
Bigr,395338,64644,330694
Adj,83422,21562,61860


# %% [markdown]

## 4. Run `ucs_to_csv()` to convert `ucs/[PAT_DIR]/readable/*.txt` to format that `pandas` can parse as a dataframe

In [ ]:
! head -5 {readable}
csv_path = ucs2csv(readable)
print(f'CSV: `{csv_path.relative_to(RESULT_DIR)}`')

         l1  l2                        f            E11  am.log.likelihood  am.odds.ratio.disc   am.p1.given2    am.p2.given1  am.p1.given2.simple  am.p2.given1.simple      f1      f2        N  
-----------  --------------------  -----  -------------  -----------------  ------------------  -------------  --------------  -------------------  -------------------  ------  ------  -------  
         or  more                  34344  46132.7018464       -4777.300943       -0.1818145104  -0.0660225433  -0.04638532985         0.1651757373        0.10515164690  326614  207924  1472077  
  something  more                  64024  44964.4617313       11177.921632        0.2471544354   0.1067428125   0.07639094924         0.3079202016        0.20111640589  318343  207924  1472077  
        all  more                  47547  35209.4943376        5664.083926        0.1935052280   0.0690961153   0.05958232948         0.2286749004        0.19073808865  249279  207924  1472077  
UCS table text converted 

# %% [markdown]

##

In [ ]:
trig_amdf = pd.read_csv(csv_path).convert_dtypes()
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
0,or,more,34344,"46,132.70","-4,777.30",-0.18,-0.07,-0.05,0.17,0.11,326614,207924,1472077
1,something,more,64024,"44,964.46","11,177.92",0.25,0.11,0.08,0.31,0.20,318343,207924,1472077
2,all,more,47547,"35,209.49","5,664.08",0.19,0.07,0.06,0.23,0.19,249279,207924,1472077
3,some,more,15949,"16,233.18",-6.31,-0.01,-0.00,-0.00,0.08,0.14,114929,207924,1472077
4,often,more,15169,"13,027.48",420.10,0.08,0.01,0.02,0.07,0.16,92233,207924,1472077
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8329,all,big-rig,3,0.51,10.66,1.54,0.83,0.00,1.00,0.00,249279,3,1472077
8330,all,conscientiously,3,0.51,10.66,1.54,0.83,0.00,1.00,0.00,249279,3,1472077
8331,all,fantasticly,3,0.51,10.66,1.54,0.83,0.00,1.00,0.00,249279,3,1472077
8332,often,eye-crossingly,3,0.19,16.62,2.02,0.94,0.00,1.00,0.00,92233,3,1472077


In [ ]:
# %%

trig_amdf['key'] = (trig_amdf.l1 + '~' +
                    trig_amdf.l2).astype('string')
trig_amdf = trig_amdf.set_index('key')
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
key,,,,,,,,,,,,,
or~more,or,more,34344,"46,132.70","-4,777.30",-0.18,-0.07,-0.05,0.17,0.11,326614,207924,1472077
something~more,something,more,64024,"44,964.46","11,177.92",0.25,0.11,0.08,0.31,0.20,318343,207924,1472077
all~more,all,more,47547,"35,209.49","5,664.08",0.19,0.07,0.06,0.23,0.19,249279,207924,1472077
some~more,some,more,15949,"16,233.18",-6.31,-0.01,-0.00,-0.00,0.08,0.14,114929,207924,1472077
often~more,often,more,15169,"13,027.48",420.10,0.08,0.01,0.02,0.07,0.16,92233,207924,1472077
...,...,...,...,...,...,...,...,...,...,...,...,...,...
all~big-rig,all,big-rig,3,0.51,10.66,1.54,0.83,0.00,1.00,0.00,249279,3,1472077
all~conscientiously,all,conscientiously,3,0.51,10.66,1.54,0.83,0.00,1.00,0.00,249279,3,1472077
all~fantasticly,all,fantasticly,3,0.51,10.66,1.54,0.83,0.00,1.00,0.00,249279,3,1472077


# %% [markdown]

## 6. Save to `./results/assoc_df/`

In [ ]:
df_csv_path = AM_DF_DIR.joinpath(
    str(csv_path.relative_to(UCS_DIR))
    .replace('/readable', '')
    .replace('.rsort-view_am-only', ''))

if not df_csv_path.is_file():
    confirm_dir(df_csv_path.parent)
    trig_amdf.to_csv(df_csv_path)

df_pkl_path = df_csv_path.with_suffix('.pkl.gz')
if not df_pkl_path.is_file():
    trig_amdf.to_pickle(df_csv_path.with_suffix('.pkl.gz'))

# %% [markdown]

## 7. Add additional AM via `add_extra_am()`

In [ ]:
ex_trig_amdf = add_extra_am(df=trig_amdf,
                            verbose=True,
                            vocab=VOCAB,
                            metrics=['t_score', 'mutual_information']
                            ).convert_dtypes()


Preview of Extended Measures (rounded)

| key             |   t_score |   mutual_information |   deltaP_min |   deltaP_max |   deltaP_max_abs |   deltaP_product |   unexpected_f |   unexpected_ratio |
|:----------------|----------:|---------------------:|-------------:|-------------:|-----------------:|-----------------:|---------------:|-------------------:|
| or~more         |    -63.61 |                -0.13 |        -0.07 |        -0.05 |             0.07 |             0.00 |     -11,788.70 |              -0.34 |
| something~more  |     75.33 |                 0.15 |         0.08 |         0.11 |             0.11 |             0.01 |      19,059.54 |               0.30 |
| all~more        |     56.58 |                 0.13 |         0.06 |         0.07 |             0.07 |             0.00 |      12,337.51 |               0.26 |
| some~more       |     -2.25 |                -0.01 |        -0.00 |        -0.00 |             0.00 |             0.00 |        -284.18 |              -

#%% [markdown]

 Add trigger features as columns: polarity and quantification type

In [ ]:
def add_feature(triggers:pd.Series, 
                group_dict: dict) -> pd.Series:
    assign_dict = invert_set_dict(group_dict)
    return triggers.apply(lambda x: assign_dict[x] if x in assign_dict.keys() else '')

ex_trig_amdf['polarity'] = add_feature(ex_trig_amdf.l1, TRIGGER_POLARITY)
ex_trig_amdf['quant'] = add_feature(ex_trig_amdf.l1, TRIGGER_QUANT)

# %% [markdown]

 Save extended AM tables to `extra/` subdirectory if not already saved

In [ ]:
df_extra_csv = df_csv_path.parent / 'extra' / \
    df_csv_path.name.replace('.csv', '_extra.csv')
print(df_extra_csv)
if not df_extra_csv.is_file():
    confirm_dir(df_extra_csv.parent)
    ex_trig_amdf.to_csv(df_extra_csv)

df_extra_pkl = df_extra_csv.with_suffix('.pkl.gz')
if not df_extra_pkl.is_file():
    ex_trig_amdf.to_pickle(df_extra_pkl)

/share/compling/projects/sanpi/results/assoc_df/trigger_eval/POSmirror/extra/TrigAdv_frq-thrMIN-7.35f_min3x_extra.csv


In [ ]:
# %%

ex_trig_full = ex_trig_amdf.copy()
ex_trig_abbr = adjust_assoc_columns(
    ex_trig_amdf[[c for c in ['polarity', 'quant'] + FOCUS if c in ex_trig_amdf.columns]]).sort_values('LRC', ascending=False)
cols = ex_trig_abbr.columns

# %% [markdown]

 Define lexical items with given lean shown in binary environment evaluation

In [ ]:
pos_prone = {
    'Adj': [
        'unrelated',
        'unable',
        'akin',
        'larger',
        'different',
        'familiar',
        'similar',
        'likely', 
        'brief', 
        'unaware'
    ],
    'Adv': [
        'slightly', 
        'definitely',
        'utterly',
        # LRC top
        'pretty', 
        'rather', 
        'plain', 
        'fairly', 
        'somewhat', 
        'otherwise', 
        'downright',
        'relatively', 
        # G2 top
        # 'very', 
        # 'even', 
        # 'just', 
        # dP1 top (and odds ratio disc)
        'plain', 
        'maybe'
    ], 
    'Bigr': [
        # G2 top
        'completely_different', 
        'too_familiar',
        'even_better',
        # dP1 top
        'quite_different', 
        'too_real', 
        'well_aware', 
        # LRC top
        'too_common', 
        'entirely_different'
        ]}
neg_prone = {
    'Bigr': [
        # LRC top
        'quite_sure',
        'really_sure', 
        'too_early', 
        'too_pleased', 
        'too_fancy',
        # dP1 top
        'entirely_sure', 
        'ever_easy', 
        'ever_perfect', 
        'particularly_surprising',
        'particularly_new',
        # G2 top
        'too_late', 
        'more_important',
        'so_easy', 
        'as_good', 
        'too_old'
        ], 
    'Adv': [
        'yet',
        #LRC top
        'ever',
        'any', 
        'longer', 
        'necessarily',
        'that',
        #dP1 top
        'before', 
        'wise',  #? How is this used as an adverb?
        'earthly',
        'remotely',
        'exactly',
        # G2 top
        'particularly', 
        'too',
        # 'inherently'
        ],
    'Adj': [
        # LRC top
        'early', 
        'late', 
        'fancy', 
        'alone', 
        'sure',
        # dP1 top
        'shabby',
        'demoralizing',
        'alone',
        'aggravating',
        'groundbreaking',
        'eventful',
        #G2 top
        'important', 
        'frustrating',
        'evident', 
        'certain'
        ]
    }
def sort_prone_by_f2(prone_list, amdf):
    return amdf.copy().loc[amdf.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
pos_prone[UNIT] = sort_prone_by_f2(pos_prone[UNIT], ex_trig_abbr)
neg_prone[UNIT] = sort_prone_by_f2(neg_prone[UNIT], ex_trig_abbr)
pos_prone[UNIT]

['pretty',
 'rather',
 'slightly',
 'otherwise',
 'fairly',
 'relatively',
 'plain',
 'downright',
 'somewhat',
 'maybe',
 'utterly',
 'definitely']

# %% [markdown]

## Strongest associations for each polarity by metric

In [ ]:
def show_metric_top(amdf: pd.DataFrame,
                    metric: str,
                    k=5, cols=[None]):
    if not any(cols):
        cols = amdf.columns
    return (pd.concat((polar_df.nlargest(k, metric)
                       for pol, polar_df in amdf.groupby('polarity')))
            .loc[:, [metric] + cols[cols != metric].to_list()]
            .reset_index(drop=True).set_index(['polarity', 'l1', 'l2'])
            )
def update_prone(exdf, pos_prone, neg_prone) -> dict: 
    prone_dict = {'positive':pos_prone, 
                  'negative': neg_prone}
    for polarity, prone in prone_dict.items():
        try: 
            polar_ex = exdf.loc[polarity, :]
        except KeyError: 
            continue
        else: 
                
            for topw in polar_ex.reset_index().l2.head(2).squeeze(): 
                if topw not in prone[UNIT]: 
                    prone[UNIT].append(topw)
            prone_dict[polarity] = prone
    return prone_dict['positive'], prone_dict['negative']

# %% [markdown]

 Top consevative log ratio $LRC$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr,"LRC")
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

LRC        quant     f  unexp_f  dP1  dP2  \
polarity l1          l2                                                       
         everythings all         5.67                  9     8.94 0.00 0.56   
         somethin    n't         1.51                  6     5.66 0.00 0.23   
         alls        not         0.00                  3     2.42 0.00 0.16   
         somethin    more        0.00                  5     1.47 0.00 0.06   
         everythings so          0.00                  4     2.86 0.00 0.18   
positive or          otherwise   6.37  existential  6378 4,922.07 0.75 0.02   
         everything  humanly     6.23    universal   256   247.54 0.79 0.01   
         every       so-slightly 5.82    universal     4     4.00 1.00 0.01   
         all         around      5.52    universal   934   768.05 0.78 0.00   
         everything  all         5.21    universal  2819 2,678.71 0.51 0.07   

                                  dP1_simple  dP2_simple        G2        N  \
polarity l1          l2                                                       
         everythings all                0.00        0.56     79.67  1472077   
         somethin    n't                0.00        0.24     24.59  1472077   
         alls        not                0.00        0.20      5.48  1472077   
         somethin    more               0.00        0.20      0.64  1472077   
         everythings so                 0.00        0.25      4.93  1472077   
positive or          otherwise          0.97        0.02 17,716.43  1472077   
         everything  humanly            0.81        0.01  1,553.24  1472077   
         every       so-slightly        1.00        0.01     67.15  1472077   
         all         around             0.95        0.00  2,966.00  1472077   
         everything  all                0.54        0.07 13,506.99  1472077   

                                      f1      f2    exp_f  
polarity l1          l2                                    
         everythings all              16    5226     0.06  
         somethin    n't              25   19938     0.34  
         alls        not              15   56607     0.58  
         somethin    more             25  207924     3.53  
         everythings so               16  104617     1.14  
positive or          otherwise    326614    6562 1,455.93  
         everything  humanly       39516     315     8.46  
         every       so-slightly     335       4     0.00  
         all         around       249279     980   165.95  
         everything  all           39516    5226   140.29

# %% [markdown]

 Top $\Delta P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1             quant  f  unexp_f  LRC  dP2  \
polarity l1          l2                                                         
         everythings all         0.00                    9     8.94 5.67 0.56   
         somethin    n't         0.00                    6     5.66 1.51 0.23   
         alls        not         0.00                    3     2.42 0.00 0.16   
         everythings so          0.00                    4     2.86 0.00 0.18   
         everyones   so          0.00                    4     2.86 0.00 0.18   
positive every       so-slightly 1.00         universal  4     4.00 5.82 0.01   
         everything  peachy      0.97         universal  4     3.89 0.00 0.00   
                     pixel       0.97         universal  3     2.92 0.00 0.00   
         always      bitter      0.94         universal  4     3.78 0.00 0.00   
         often       drily       0.94  hedged_universal  4     3.75 0.00 0.00   

                                  dP1_simple  dP2_simple    G2        N  \
polarity l1          l2                                                   
         everythings all                0.00        0.56 79.67  1472077   
         somethin    n't                0.00        0.24 24.59  1472077   
         alls        not                0.00        0.20  5.48  1472077   
         everythings so                 0.00        0.25  4.93  1472077   
         everyones   so                 0.00        0.25  4.93  1472077   
positive every       so-slightly        1.00        0.01 67.15  1472077   
         everything  peachy             1.00        0.00 28.94  1472077   
                     pixel              1.00        0.00 21.71  1472077   
         always      bitter             1.00        0.00 23.08  1472077   
         often       drily              1.00        0.00 22.16  1472077   

                                     f1      f2  exp_f  
polarity l1          l2                                 
         everythings all             16    5226   0.06  
         somethin    n't             25   19938   0.34  
         alls        not             15   56607   0.58  
         everythings so              16  104617   1.14  
         everyones   so              16  104617   1.14  
positive every       so-slightly    335       4   0.00  
         everything  peachy       39516       4   0.11  
                     pixel        39516       3   0.08  
         always      bitter       82200       4   0.22  
         often       drily        92233       4   0.25

# %% [markdown]

 Top conditional probability $P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1_simple        quant   f  unexp_f  LRC  \
polarity l1          l2                                                         
         everythings all                  0.00                9     8.94 5.67   
         somethin    n't                  0.00                6     5.66 1.51   
         alls        not                  0.00                3     2.42 0.00   
         everythings so                   0.00                4     2.86 0.00   
         everyones   so                   0.00                4     2.86 0.00   
positive every       so-slightly          1.00    universal   4     4.00 5.82   
         something   sizey                1.00  existential  80    62.70 0.72   
                     primally             1.00  existential  13    10.19 0.00   
         or          second-most          1.00  existential   6     4.67 0.00   
                     etiologically        1.00  existential   6     4.67 0.00   

                                    dP1  dP2  dP2_simple     G2        N  \
polarity l1          l2                                                    
         everythings all           0.00 0.56        0.56  79.67  1472077   
         somethin    n't           0.00 0.23        0.24  24.59  1472077   
         alls        not           0.00 0.16        0.20   5.48  1472077   
         everythings so            0.00 0.18        0.25   4.93  1472077   
         everyones   so            0.00 0.18        0.25   4.93  1472077   
positive every       so-slightly   1.00 0.01        0.01  67.15  1472077   
         something   sizey         0.78 0.00        0.00 245.02  1472077   
                     primally      0.78 0.00        0.00  39.81  1472077   
         or          second-most   0.78 0.00        0.00  18.07  1472077   
                     etiologically 0.78 0.00        0.00  18.07  1472077   

                                        f1      f2  exp_f  
polarity l1          l2                                    
         everythings all                16    5226   0.06  
         somethin    n't                25   19938   0.34  
         alls        not                15   56607   0.58  
         everythings so                 16  104617   1.14  
         everyones   so                 16  104617   1.14  
positive every       so-slightly       335       4   0.00  
         something   sizey          318343      80  17.30  
                     primally       318343      13   2.81  
         or          second-most    326614       6   1.33  
                     etiologically  326614       6   1.33

# %% [markdown]

 Top $\Delta P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2        quant      f   unexp_f  LRC  dP1  \
polarity l1          l2                                                  
         everythings all  0.56                   9      8.94 5.67 0.00   
         somethin    n't  0.23                   6      5.66 1.51 0.00   
         everythings so   0.18                   4      2.86 0.00 0.00   
         everyones   so   0.18                   4      2.86 0.00 0.00   
         alls        not  0.16                   3      2.42 0.00 0.00   
positive either      too  0.21  existential   1110    814.32 1.73 0.01   
         everyone    so   0.18    universal   8043  5,745.60 1.82 0.06   
         everybody   so   0.17    universal   1894  1,327.95 1.61 0.01   
         always      very 0.13    universal  19988 10,210.23 1.07 0.07   
         or          even 0.13  existential  45543 32,647.52 3.59 0.58   

                           dP1_simple  dP2_simple        G2        N      f1  \
polarity l1          l2                                                        
         everythings all         0.00        0.56     79.67  1472077      16   
         somethin    n't         0.00        0.24     24.59  1472077      25   
         everythings so          0.00        0.25      4.93  1472077      16   
         everyones   so          0.00        0.25      4.93  1472077      16   
         alls        not         0.00        0.20      5.48  1472077      15   
positive either      too         0.01        0.29  1,514.98  1472077    3886   
         everyone    so          0.08        0.25 10,195.44  1472077   32327   
         everybody   so          0.02        0.24  2,191.44  1472077    7965   
         always      very        0.11        0.24 10,404.38  1472077   82200   
         or          even        0.78        0.14 87,219.31  1472077  326614   

                               f2     exp_f  
polarity l1          l2                      
         everythings all     5226      0.06  
         somethin    n't    19938      0.34  
         everythings so    104617      1.14  
         everyones   so    104617      1.14  
         alls        not    56607      0.58  
positive either      too   112009    295.68  
         everyone    so    104617  2,297.40  
         everybody   so    104617    566.05  
         always      very  175105  9,777.77  
         or          even   58121 12,895.48

# %% [markdown]

 Top conditional probability $P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2_simple        quant      f   unexp_f  LRC  dP1  \
polarity l1          l2                                                         
         everythings all         0.56                   9      8.94 5.67 0.00   
                     so          0.25                   4      2.86 0.00 0.00   
         everyones   so          0.25                   4      2.86 0.00 0.00   
         somethin    n't         0.24                   6      5.66 1.51 0.00   
         alls        not         0.20                   3      2.42 0.00 0.00   
positive either      too         0.29  existential   1110    814.32 1.73 0.01   
         everyone    so          0.25    universal   8043  5,745.60 1.82 0.06   
         always      very        0.24    universal  19988 10,210.23 1.07 0.07   
         everybody   so          0.24    universal   1894  1,327.95 1.61 0.01   
         somethings  more        0.21  existential     33     10.82 0.00 0.00   

                           dP2  dP1_simple        G2        N     f1      f2  \
polarity l1          l2                                                        
         everythings all  0.56        0.00     79.67  1472077     16    5226   
                     so   0.18        0.00      4.93  1472077     16  104617   
         everyones   so   0.18        0.00      4.93  1472077     16  104617   
         somethin    n't  0.23        0.00     24.59  1472077     25   19938   
         alls        not  0.16        0.00      5.48  1472077     15   56607   
positive either      too  0.21        0.01  1,514.98  1472077   3886  112009   
         everyone    so   0.18        0.08 10,195.44  1472077  32327  104617   
         always      very 0.13        0.11 10,404.38  1472077  82200  175105   
         everybody   so   0.17        0.02  2,191.44  1472077   7965  104617   
         somethings  more 0.07        0.00      5.48  1472077    157  207924   

                             exp_f  
polarity l1          l2             
         everythings all      0.06  
                     so       1.14  
         everyones   so       1.14  
         somethin    n't      0.34  
         alls        not      0.58  
positive either      too    295.68  
         everyone    so   2,297.40  
         always      very 9,777.77  
         everybody   so     566.05  
         somethings  more    22.18

# %% [markdown]

 Top log-likelihood $G^2$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'G2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

G2        quant      f   unexp_f  LRC  \
polarity l1          l2                                                       
         everythings all           79.67                   9      8.94 5.67   
         somethin    n't           24.59                   6      5.66 1.51   
         alls        not            5.48                   3      2.42 0.00   
         everythings so             4.93                   4      2.86 0.00   
         everyones   so             4.93                   4      2.86 0.00   
positive or          even      87,219.31  existential  45543 32,647.52 3.59   
         all         too       38,448.80    universal  45382 26,414.59 1.70   
         or          otherwise 17,716.43  existential   6378  4,922.07 6.37   
         something   as        14,184.17  existential  30864 14,215.23 1.23   
         or          just      13,554.55  existential  14940  8,810.76 1.96   

                                dP1  dP2  dP1_simple  dP2_simple        N  \
polarity l1          l2                                                     
         everythings all       0.00 0.56        0.00        0.56  1472077   
         somethin    n't       0.00 0.23        0.00        0.24  1472077   
         alls        not       0.00 0.16        0.00        0.20  1472077   
         everythings so        0.00 0.18        0.00        0.25  1472077   
         everyones   so        0.00 0.18        0.00        0.25  1472077   
positive or          even      0.58 0.13        0.78        0.14  1472077   
         all         too       0.26 0.13        0.41        0.18  1472077   
         or          otherwise 0.75 0.02        0.97        0.02  1472077   
         something   as        0.19 0.06        0.40        0.10  1472077   
         or          just      0.33 0.03        0.54        0.05  1472077   

                                    f1      f2     exp_f  
polarity l1          l2                                   
         everythings all            16    5226      0.06  
         somethin    n't            25   19938      0.34  
         alls        not            15   56607      0.58  
         everythings so             16  104617      1.14  
         everyones   so             16  104617      1.14  
positive or          even       326614   58121 12,895.48  
         all         too        249279  112009 18,967.41  
         or          otherwise  326614    6562  1,455.93  
         something   as         318343   76987 16,648.77  
         or          just       326614   27625  6,129.24

In [ ]:
# %%

sig_trig_abbr = ex_trig_abbr.loc[ex_trig_abbr.LRC.abs() > 1, :]
def show_prone_ex(amdf, prone_list, list_index):
    try: 
        example = prone[list_index]

    except IndexError: 
        return list_index + 1, f'No {list_index}th entry'
    else:
        print(f'>> {example} <<')
        return list_index+ 1, amdf.filter(regex=f'~{example}$', axis=0).iloc[:10, :]

# %% [markdown]

 ---
 Significant Examples for Items demonstrated positive polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone = pos_prone[UNIT]
prone

['pretty',
 'rather',
 'slightly',
 'otherwise',
 'fairly',
 'relatively',
 'plain',
 'downright',
 'somewhat',
 'maybe',
 'utterly',
 'definitely',
 'humanly',
 'so-slightly',
 'peachy',
 'sizey',
 'too',
 'so',
 'even']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> pretty <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everybody~pretty,positive,universal,421,287.90,1.32,0.01,0.04,0.02,0.05,407.97,1472077,7965,24599,133.10,everybody,pretty
some~pretty,positive,existential,4085,"2,164.49",1.11,0.09,0.02,0.17,0.04,"2,095.89",1472077,114929,24599,"1,920.51",some,pretty
all~pretty,positive,universal,7720,"3,554.45",1.07,0.15,0.02,0.31,0.03,"3,136.99",1472077,249279,24599,"4,165.55",all,pretty
someone~pretty,positive,existential,147,-348.66,-1.16,-0.01,-0.01,0.01,0.00,-349.24,1472077,29662,24599,495.66,someone,pretty
or~pretty,positive,existential,1099,"-4,358.85",-2.38,-0.18,-0.02,0.04,0.00,"-6,195.67",1472077,326614,24599,"5,457.85",or,pretty


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> rather <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> slightly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
or~slightly,positive,existential,3411,"1,741.63",1.37,0.23,0.01,0.45,0.01,"1,982.29",1472077,326614,7524,"1,669.37",or,slightly
all~slightly,positive,universal,452,-822.10,-1.31,-0.11,-0.00,0.06,0.00,-814.34,1472077,249279,7524,"1,274.10",all,slightly
many~slightly,positive,hedged_universal,34,-206.90,-1.59,-0.03,-0.00,0.00,0.00,-287.43,1472077,47133,7524,240.90,many,slightly


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> otherwise <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
or~otherwise,positive,existential,6378,"4,922.07",6.37,0.75,0.02,0.97,0.02,"17,716.43",1472077,326614,6562,"1,455.93",or,otherwise
someone~otherwise,positive,existential,20,-112.22,-1.09,-0.02,-0.00,0.00,0.00,-151.28,1472077,29662,6562,132.22,someone,otherwise
everyone~otherwise,positive,universal,3,-141.10,-1.34,-0.02,-0.00,0.00,0.00,-262.68,1472077,32327,6562,144.10,everyone,otherwise
many~otherwise,positive,hedged_universal,26,-184.10,-1.60,-0.03,-0.00,0.00,0.00,-265.58,1472077,47133,6562,210.10,many,otherwise
everything~otherwise,positive,universal,6,-170.15,-1.89,-0.03,-0.00,0.00,0.00,-304.99,1472077,39516,6562,176.15,everything,otherwise
both~otherwise,positive,universal,6,-216.91,-2.24,-0.03,-0.00,0.00,0.00,-398.76,1472077,50007,6562,222.91,both,otherwise
often~otherwise,positive,hedged_universal,25,-386.14,-2.64,-0.06,-0.00,0.00,0.00,-657.76,1472077,92233,6562,411.14,often,otherwise
sometimes~otherwise,positive,existential,5,-328.02,-2.81,-0.05,-0.00,0.00,0.00,-632.55,1472077,74707,6562,333.02,sometimes,otherwise
some~otherwise,positive,existential,20,-492.31,-3.13,-0.08,-0.00,0.00,0.00,-896.21,1472077,114929,6562,512.31,some,otherwise


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> fairly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
both~fairly,positive,universal,483,290.12,1.05,0.05,0.01,0.09,0.01,323.86,1472077,50007,5678,192.88,both,fairly
or~fairly,positive,existential,565,-694.79,-1.04,-0.12,-0.00,0.10,0.00,-589.32,1472077,326614,5678,"1,259.79",or,fairly


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> relatively <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
both~relatively,positive,universal,465,284.01,1.08,0.05,0.01,0.09,0.01,327.16,1472077,50007,5328,180.99,both,relatively


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> plain <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
or~plain,positive,existential,3681,"2,559.88",3.00,0.51,0.01,0.73,0.01,"5,889.24",1472077,326614,5053,"1,121.12",or,plain
someone~plain,positive,existential,9,-92.82,-1.05,-0.02,-0.00,0.00,0.00,-143.99,1472077,29662,5053,101.82,someone,plain
everyone~plain,positive,universal,9,-101.96,-1.18,-0.02,-0.00,0.00,0.00,-161.13,1472077,32327,5053,110.96,everyone,plain
everything~plain,positive,universal,18,-117.64,-1.20,-0.02,-0.00,0.00,0.00,-165.73,1472077,39516,5053,135.64,everything,plain
both~plain,positive,universal,5,-166.65,-1.83,-0.03,-0.00,0.00,0.00,-304.15,1472077,50007,5053,171.65,both,plain
always~plain,positive,universal,16,-266.16,-2.36,-0.05,-0.00,0.00,0.00,-455.97,1472077,82200,5053,282.16,always,plain
all~plain,positive,universal,67,-788.67,-3.01,-0.16,-0.00,0.01,0.00,"-1,378.71",1472077,249279,5053,855.67,all,plain
something~plain,positive,existential,56,"-1,036.73",-3.63,-0.21,-0.00,0.01,0.00,"-1,995.43",1472077,318343,5053,"1,092.73",something,plain


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> downright <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
sometimes~downright,positive,existential,1034,793.96,2.13,0.17,0.01,0.22,0.01,"1,590.56",1472077,74707,4730,240.04,sometimes,downright
some~downright,positive,existential,1279,909.72,1.89,0.19,0.01,0.27,0.01,"1,570.70",1472077,114929,4730,369.28,some,downright
everything~downright,positive,universal,21,-105.97,-1.01,-0.02,-0.00,0.00,0.00,-139.08,1472077,39516,4730,126.97,everything,downright
everyone~downright,positive,universal,6,-97.87,-1.12,-0.02,-0.00,0.00,0.00,-163.89,1472077,32327,4730,103.87,everyone,downright
both~downright,positive,universal,9,-151.68,-1.73,-0.03,-0.00,0.00,0.00,-256.94,1472077,50007,4730,160.68,both,downright
always~downright,positive,universal,7,-257.12,-2.51,-0.05,-0.00,0.00,0.00,-478.80,1472077,82200,4730,264.12,always,downright
something~downright,positive,existential,132,-890.88,-2.61,-0.19,-0.00,0.03,0.00,"-1,443.14",1472077,318343,4730,"1,022.88",something,downright
all~downright,positive,universal,46,-754.97,-3.28,-0.16,-0.00,0.01,0.00,"-1,386.42",1472077,249279,4730,800.97,all,downright


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> somewhat <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
always~somewhat,positive,universal,777,529.02,1.55,0.12,0.01,0.17,0.01,790.13,1472077,82200,4441,247.98,always,somewhat
something~somewhat,positive,existential,313,-647.39,-1.43,-0.15,-0.00,0.07,0.00,-708.20,1472077,318343,4441,960.39,something,somewhat


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> maybe <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
or~maybe,positive,existential,2334,"1,763.12",4.59,0.69,0.01,0.91,0.01,"5,569.58",1472077,326614,2573,570.88,or,maybe
often~maybe,positive,hedged_universal,3,-158.21,-1.56,-0.06,-0.00,0.00,0.00,-302.97,1472077,92233,2573,161.21,often,maybe
something~maybe,positive,existential,64,-492.42,-2.50,-0.19,-0.00,0.02,0.00,-820.50,1472077,318343,2573,556.42,something,maybe
all~maybe,positive,universal,7,-428.71,-3.42,-0.17,-0.00,0.00,0.00,-881.23,1472077,249279,2573,435.71,all,maybe


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> utterly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


# %% [markdown]

 ---
## Examples for Items demonstrating *Negative* polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone_list = neg_prone[UNIT]
prone = sig_trig_abbr.copy().loc[sig_trig_abbr.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
prone

['too', 'particularly', 'that', 'remotely', 'yet', 'exactly', 'any']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
either~too,positive,existential,1110,814.32,1.73,0.01,0.21,0.01,0.29,"1,514.98",1472077,3886,112009,295.68,either,too
all~too,positive,universal,45382,"26,414.59",1.70,0.26,0.13,0.41,0.18,"38,448.80",1472077,249279,112009,"18,967.41",all,too
everyone~too,positive,universal,1046,"-1,413.73",-1.03,-0.01,-0.04,0.01,0.03,"-1,124.19",1472077,32327,112009,"2,459.73",everyone,too
both~too,positive,universal,1526,"-2,278.99",-1.16,-0.02,-0.05,0.01,0.03,"-1,931.71",1472077,50007,112009,"3,804.99",both,too
everything~too,positive,universal,1040,"-1,966.74",-1.33,-0.02,-0.05,0.01,0.03,"-1,867.57",1472077,39516,112009,"3,006.74",everything,too
someone~too,positive,existential,693,"-1,563.95",-1.44,-0.02,-0.05,0.01,0.02,"-1,603.02",1472077,29662,112009,"2,256.95",someone,too
something~too,positive,existential,2079,"-22,143.43",-3.70,-0.21,-0.09,0.02,0.01,"-41,334.74",1472077,318343,112009,"24,222.43",something,too


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> particularly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
all~particularly,positive,universal,234,-410.67,-1.14,-0.11,-0.00,0.06,0.00,-399.04,1472077,249279,3807,644.67,all,particularly


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> that <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
somebody~that,positive,existential,32,29.82,2.59,0.02,0.01,0.03,0.01,113.45,1472077,2682,1195,2.18,somebody,that
someone~that,positive,existential,93,68.92,1.24,0.06,0.00,0.08,0.00,117.79,1472077,29662,1195,24.08,someone,that
something~that,positive,existential,64,-194.42,-1.33,-0.16,-0.00,0.05,0.00,-248.17,1472077,318343,1195,258.42,something,that


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> remotely <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everything~remotely,positive,universal,54,40.71,1.08,0.08,0.00,0.11,0.00,73.59,1472077,39516,495,13.29,everything,remotely


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> yet <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
something~yet,positive,existential,292,185.82,1.73,0.38,0.00,0.59,0.00,328.45,1472077,318343,491,106.18,something,yet


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> exactly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
everything~exactly,positive,universal,54,48.01,2.38,0.22,0.00,0.24,0.00,153.09,1472077,39516,223,5.99,everything,exactly


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> any <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
or~any,positive,existential,82,57.15,1.68,0.51,0.00,0.73,0.00,131.82,1472077,326614,112,24.85,or,any


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

'No 7th entry'

In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

'No 8th entry'

In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

'No 9th entry'

In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

'No 10th entry'

In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

'No 11th entry'